In [1]:
import tabulate
from pathlib import Path
import numpy as np

from typing import List, Dict
from collections import defaultdict
import warnings
import copy

warnings.filterwarnings("error")


import tabulate

log_dir = '../log'

models_map = {
    'dcgan': 'DCGAN', 
    'wgan_gp': 'WGAN-GP', 
    'sngan_ns': 'SNGAN', 
    'snresnet': 'SNGAN (R)', 
    'studio_dcgan': 'DCGAN (C)',
    'studio_sngan': 'SNGAN (C)',
    'studio_wgan_gp': 'WGAN-GP (C)',
    'studio_sagan': 'SAGAN',     
   'studio_stylegan2': 'StyleGAN2-ADA'
}

def prettify(exps_mean, exps_cp):
    for key in metrics:
        exps_cp[key][''] = []
        for k in exps_mean[key]['']:
            k = k.replace('discriminator', 'dis')
            k = k.replace('mmd', 'MMD')
            k = k.replace('cmd', 'CMD')
            k = k.replace('_dis', ' (d)')
            k = k.replace('_', ' ')
            if k == 'cluster':
                k = 'cluster v0'
            k = k.replace('dumb', 'w/o constr.')
            exps_cp[key][''].append(k)

        for model in models:
            exps_cp[key].pop(model)
            for i, mean in enumerate(exps_mean[key][model]):
                model_ = models_map[model]

                exps_cp[key][model_].append(mean)

In [2]:

seeds = {}
for model in models_map:
    seeds[model] = ['']
models = list(models_map.keys())
step_sizes = [0.01] * (len(models))
step_sizes = [0.003, 0.001, 0.003] + [0.01] * (len(models)-4) + [0.003]
metrics = ['fid_values']

features = {
    "dumb": [""],
    #"discriminator": [""],
    "cluster": {
       #  "": ["", "_dis"],
       # "_v1": ["", "_dis"],
        "_v2": {
            "": ["", "_dis"],
            # "_lin": ["", "_dis"],
            # "_quad": ["", "_dis"],
        },
        # "_v3": ["", "_dis"]
    },
    "cluster_resnet34": {
         "": [""],
        "_v1": [""],
        "_v2": {
            "": [""],
            # "_lin": ["", "_dis"],
            # "_quad": ["", "_dis"],
        },
        # "_v3": ["", "_dis"]
    },
    # "mmd": {
    #         "": ["", "_dis"],
    #         # "_lin": [""], #"_dis"],
    #         # "_quad": [""],#, "_dis"],
    #     },
    "cmd": {
            "": ["", "_dis"],
        }, 
    # "mmd_resnet34": {
    #         "": ["",],
    #         # "_lin": [""], #"_dis"],
    #         # "_quad": [""],#, "_dis"],
    #     },
    "cmd_resnet34": {
            "": ["",],
        }, 
    "resnet34": [""]
}
targets = ["Discriminator", "Prior"]

In [3]:
def collect(step):
    exps_mean = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    exps_std = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    for target in targets:
        for metric in metrics:
            exps_mean[target][metric][''].append('GAN, 10k')
            exps_mean[target][metric][''].append('GAN, 1k')

    for target in targets:
        for model, step_size in zip(models, step_sizes):
            for metric in metrics:
                arr = []
                for seed in seeds[model]:
                    path = Path(log_dir, f'dumb_feature_DiscriminatorTarget', f'{model}_{step_size}{seed}')
                    try:
                        met = np.loadtxt(Path(path, f'{metric}.txt'))[0]
                    except (IndexError, UserWarning, OSError):
                        met = None
                    if met:
                        arr.append(met)
                mean = np.mean(arr) if len(arr) > 0 else 0
                std = np.std(arr) if len(arr) > 0 else 0       
                exps_mean[target][metric][model].append(mean)
                exps_std[target][metric][model].append(std)

    for target in targets:
        for model, step_size in zip(models, step_sizes):
            for metric in metrics:
                arr = []
                for seed in seeds[model]:
                    path = Path(log_dir, f'discriminator_feature_DiscriminatorTarget', f'{model}_{step_size}{seed}')
                    try:
                        met = np.loadtxt(Path(path, f'{metric}.txt'))[0]
                    except (IndexError, UserWarning, OSError):
                        met = None
                    if met:
                        arr.append(met)
                mean = np.mean(arr) if len(arr) > 0 else 0
                std = np.std(arr) if len(arr) > 0 else 0       
                exps_mean[target][metric][model].append(mean)
                exps_std[target][metric][model].append(std)

    def move(subs, suff, feature):
        if isinstance(subs, List):
            for target in targets:
                for model, step_size in zip(models, step_sizes):
                    for s in subs:
                        for metric in metrics:
                            arr = []
                            for seed in seeds[model]:
                                path = Path(log_dir, f'{feature}_feature_{target}Target', f'{model}{suff}{s}_{step_size}{seed}')
                                if model == models[0] and seed == seeds[model][0]:
                                    exps_mean[target][metric][''].append(feature+path.name[len(model):-len(f'_{step_size}{seed}')])
                                    exps_std[target][metric][''].append(feature+path.name[len(model):-len(f'_{step_size}{seed}')])

                                try:
                                    met = np.loadtxt(Path(path, f'{metric}.txt'))[-1]#[step]
                                except (IndexError, UserWarning, OSError):
                                    met = None
                                if met:
                                    arr.append(met)
                                
                            mean = np.mean(arr) if len(arr) > 0 else 0
                            std = np.std(arr) if len(arr) > 0 else 0 
                            exps_mean[target][metric][model].append(mean)
                            exps_std[target][metric][model].append(std)
            return
            
        elif isinstance(subs, Dict):
            for sub_key, subs_ in subs.items():
                move(subs_, suff+sub_key, feature)

    for feature, subs in features.items():
        move(subs, "", feature)
    return exps_mean, exps_std




In [4]:
exps_mean, exps_std = collect(2)#20)

exps_mean_cp =  copy.deepcopy(exps_mean)
exps_std_cp =  copy.deepcopy(exps_std)

for target in targets:
    prettify(exps_mean[target], exps_mean_cp[target])
    prettify(exps_std[target], exps_std_cp[target])

exps_cp = copy.deepcopy(exps_mean_cp)

for target in targets:
    for key in metrics:
        for model in list(exps_mean_cp[target][key].keys())[1:]:
            for i, (mean, std) in enumerate(zip(exps_mean_cp[target][key][model], exps_std_cp[target][key][model])):
                exps_cp[target][key][model][i] = fr'${mean:.1f}$' #({std:.3f})$'

In [5]:

print(tabulate.tabulate(exps_cp["Discriminator"]["fid_values"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))


\begin{tabular}{llllllllll}
\hline
                     & DCGAN   & WGAN-GP   & SNGAN   & SNGAN (R)   & DCGAN (C)   & SNGAN (C)   & WGAN-GP (C)   & SAGAN   & StyleGAN2-ADA   \\
\hline
 GAN, 10k            & $35.8$  & $33.0$    & $25.7$  & $18.3$      & $47.6$      & $14.0$      & $24.5$        & $14.8$  & $9.5$           \\
 GAN, 1k             & $0.0$   & $33.3$    & $0.0$   & $0.0$       & $47.5$      & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 w/o constr.         & $41.6$  & $36.9$    & $26.3$  & $25.4$      & $47.5$      & $13.1$      & $42.9$        & $12.8$  & $9.9$           \\
 cluster v2          & $0.0$   & $0.0$     & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster v2 (d)      & $0.0$   & $0.0$     & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster resnet34    & $42.3$  & $31.6$    & $23.3$  & $20.7$      & $0.0$       & $0.0$       & $0.0$    

In [6]:
# cmd_resnet34, cmd_d, cluster_resnet34_v1, cluster_resnet34

In [7]:
print(tabulate.tabulate(exps_cp["Prior"]["fid_values"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))

\begin{tabular}{llllllllll}
\hline
                     & DCGAN   & WGAN-GP   & SNGAN   & SNGAN (R)   & DCGAN (C)   & SNGAN (C)   & WGAN-GP (C)   & SAGAN   & StyleGAN2-ADA   \\
\hline
 GAN, 10k            & $35.8$  & $33.0$    & $25.7$  & $18.3$      & $47.6$      & $14.0$      & $24.5$        & $14.8$  & $9.5$           \\
 GAN, 1k             & $0.0$   & $33.3$    & $0.0$   & $0.0$       & $47.5$      & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 w/o constr.         & $0.0$   & $0.0$     & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster v2          & $0.0$   & $0.0$     & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster v2 (d)      & $34.3$  & $33.5$    & $23.0$  & $15.5$      & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster resnet34    & $0.0$   & $32.0$    & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$    

In [65]:
# cmd_resnet34, cmd_d, cluster v2 (d)

In [9]:

seeds = {}
for model in models_map:
    seeds[model] = ['']
models = list(models_map.keys())
step_sizes = [0.003, 0.001, 0.003] + [0.01] * (len(models) - 3)
metrics = ['fid_values']

features = {
    "dumb": [""],
    #"discriminator": [""],
    "cluster": {
       #  "": ["", "_dis"],
       # "_v1": ["", "_dis"],
        "_v2": {
            "": ["", "_dis"],
            # "_lin": ["", "_dis"],
            # "_quad": ["", "_dis"],
        },
        # "_v3": ["", "_dis"]
    },
    "cluster_resnet34": {
         "": [""],
        "_v1": [""],
        "_v2": {
            "": [""],
            # "_lin": ["", "_dis"],
            # "_quad": ["", "_dis"],
        },
        # "_v3": ["", "_dis"]
    },
    # "mmd": {
    #         "": ["", "_dis"],
    #         # "_lin": [""], #"_dis"],
    #         # "_quad": [""],#, "_dis"],
    #     },
    "cmd": {
            "": ["", "_dis"],
        }, 
    # "mmd_resnet34": {
    #         "": ["",],
    #         # "_lin": [""], #"_dis"],
    #         # "_quad": [""],#, "_dis"],
    #     },
    "cmd_resnet34": {
            "": ["",],
        }, 
    "resnet34": [""]
}
targets = ["Discriminator", "Prior"]

In [10]:
exps_mean, exps_std = collect(2)#20)

exps_mean_cp =  copy.deepcopy(exps_mean)
exps_std_cp =  copy.deepcopy(exps_std)

for target in targets:
    prettify(exps_mean[target], exps_mean_cp[target])
    prettify(exps_std[target], exps_std_cp[target])

exps_cp = copy.deepcopy(exps_mean_cp)

for target in targets:
    for key in metrics:
        for model in list(exps_mean_cp[target][key].keys())[1:]:
            for i, (mean, std) in enumerate(zip(exps_mean_cp[target][key][model], exps_std_cp[target][key][model])):
                exps_cp[target][key][model][i] = fr'${mean:.1f}$' #({std:.3f})$'

In [11]:
print(tabulate.tabulate(exps_cp["Discriminator"]["fid_values"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))


\begin{tabular}{llllllllll}
\hline
                     & DCGAN   & WGAN-GP   & SNGAN   & SNGAN (R)   & DCGAN (C)   & SNGAN (C)   & WGAN-GP (C)   & SAGAN   & StyleGAN2-ADA   \\
\hline
 GAN, 10k            & $35.8$  & $33.0$    & $25.7$  & $18.3$      & $47.6$      & $14.0$      & $24.5$        & $14.8$  & $9.6$           \\
 GAN, 1k             & $0.0$   & $33.3$    & $0.0$   & $0.0$       & $47.5$      & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 w/o constr.         & $41.6$  & $36.9$    & $26.3$  & $25.4$      & $47.5$      & $13.1$      & $42.9$        & $12.8$  & $11.8$          \\
 cluster v2          & $0.0$   & $0.0$     & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster v2 (d)      & $0.0$   & $0.0$     & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster resnet34    & $42.3$  & $31.6$    & $23.3$  & $20.7$      & $0.0$       & $0.0$       & $0.0$    

In [12]:
print(tabulate.tabulate(exps_cp["Prior"]["fid_values"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))


\begin{tabular}{llllllllll}
\hline
                     & DCGAN   & WGAN-GP   & SNGAN   & SNGAN (R)   & DCGAN (C)   & SNGAN (C)   & WGAN-GP (C)   & SAGAN   & StyleGAN2-ADA   \\
\hline
 GAN, 10k            & $35.8$  & $33.0$    & $25.7$  & $18.3$      & $47.6$      & $14.0$      & $24.5$        & $14.8$  & $9.6$           \\
 GAN, 1k             & $0.0$   & $33.3$    & $0.0$   & $0.0$       & $47.5$      & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 w/o constr.         & $0.0$   & $0.0$     & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster v2          & $0.0$   & $0.0$     & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster v2 (d)      & $34.3$  & $33.5$    & $23.0$  & $15.5$      & $0.0$       & $0.0$       & $0.0$         & $0.0$   & $0.0$           \\
 cluster resnet34    & $0.0$   & $32.0$    & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$    

In [13]:
models_map = {
    'mmc_sngan': 'SNGAN'
}

seeds = {}
for model in models_map:
    seeds[model] = ['']
models = list(models_map.keys())
step_sizes = [0.01] * len(models)
metrics = ['fid_values']

features = {
    "dumb": [""],
    "discriminator": [""],
    "cluster": {
         "": ["", "_dis"],
        "_v1": ["", "_dis"],
        "_v2": {
            "": ["", "_dis"],
            # "_lin": ["", "_dis"],
            # "_quad": ["", "_dis"],
        },
        # "_v3": ["", "_dis"]
    },
    "cluster_resnet34": {
         "": [""],
        "_v1": [""],
        "_v2": {
            "": [""],
            # "_lin": ["", "_dis"],
            # "_quad": ["", "_dis"],
        },
        # "_v3": ["", "_dis"]
    },
    "mmd": {
            "": ["", "_dis"],
            # "_lin": [""], #"_dis"],
            # "_quad": [""],#, "_dis"],
        },
    "cmd": {
            "": ["", "_dis"],
        }, 
    "mmd_resnet34": {
            "": ["",],
            # "_lin": [""], #"_dis"],
            # "_quad": [""],#, "_dis"],
        },
    "cmd_resnet34": {
            "": ["",],
        }, 
    "resnet34": [""]
}
targets = ["Discriminator", "Prior"]

In [14]:
exps_mean = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
exps_std = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for target in targets:
    for metric in metrics:
        exps_mean[target][metric][''].append('GAN')

for target in targets:
    for model, step_size in zip(models, step_sizes):
        for metric in metrics:
            arr = []
            for seed in seeds[model]:
                path = Path(log_dir, f'dumb_feature_DiscriminatorTarget', f'{model}_{step_size}{seed}')
                try:
                    met = np.loadtxt(Path(path, f'{metric}.txt'))[0]
                except (IndexError, UserWarning, OSError):
                    met = None
                if met:
                    arr.append(met)
            mean = np.mean(arr) if len(arr) > 0 else 0
            std = np.std(arr) if len(arr) > 0 else 0       
            exps_mean[target][metric][model].append(mean)
            exps_std[target][metric][model].append(std)

def move(subs, suff, feature):
    if isinstance(subs, List):
        for target in targets:
            for model, step_size in zip(models, step_sizes):
                for s in subs:
                    for metric in metrics:
                        arr = []
                        for seed in seeds[model]:
                            path = Path(log_dir, f'{feature}_feature_{target}Target', f'{model}{suff}{s}_{step_size}{seed}')
                            if model == models[0] and seed == seeds[model][0]:
                                exps_mean[target][metric][''].append(feature+path.name[len(model):-len(f'_{step_size}{seed}')])
                                exps_std[target][metric][''].append(feature+path.name[len(model):-len(f'_{step_size}{seed}')])

                            try:
                                met = np.loadtxt(Path(path, f'{metric}.txt'))[-1]
                            except (IndexError, UserWarning, OSError):
                                met = None
                            if met:
                                arr.append(met)
                            
                        mean = np.mean(arr) if len(arr) > 0 else 0
                        std = np.std(arr) if len(arr) > 0 else 0 
                        exps_mean[target][metric][model].append(mean)
                        exps_std[target][metric][model].append(std)
        return
        
    elif isinstance(subs, Dict):
        for sub_key, subs_ in subs.items():
            move(subs_, suff+sub_key, feature)

for feature, subs in features.items():
    move(subs, "", feature)


In [15]:
exps_mean_cp =  copy.deepcopy(exps_mean)
exps_std_cp =  copy.deepcopy(exps_std)

for target in targets:
    prettify(exps_mean[target], exps_mean_cp[target])
    prettify(exps_std[target], exps_std_cp[target])

exps_cp = copy.deepcopy(exps_mean_cp)

for target in targets:
    for key in metrics:
        for model in list(exps_mean_cp[target][key].keys())[1:]:
            for i, (mean, std) in enumerate(zip(exps_mean_cp[target][key][model], exps_std_cp[target][key][model])):
                exps_cp[target][key][model][i] = fr'${mean:.1f}$' #({std:.3f})$'

In [16]:
print(tabulate.tabulate(exps_cp["Discriminator"]["fid_values"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))


\begin{tabular}{ll}
\hline
                     & SNGAN   \\
\hline
 GAN                 & $6.5$   \\
 w/o constr.         & $6.4$   \\
 dis                 & $0.0$   \\
 cluster v0          & $0.0$   \\
 cluster (d)         & $0.0$   \\
 cluster v1          & $0.0$   \\
 cluster v1 (d)      & $0.0$   \\
 cluster v2          & $0.0$   \\
 cluster v2 (d)      & $0.0$   \\
 cluster resnet34    & $0.0$   \\
 cluster resnet34 v1 & $0.0$   \\
 cluster resnet34 v2 & $0.0$   \\
 MMD                 & $0.0$   \\
 MMD (d)             & $0.0$   \\
 CMD                 & $0.0$   \\
 CMD (d)             & $0.0$   \\
 MMD resnet34        & $0.0$   \\
 CMD resnet34        & $0.0$   \\
 resnet34            & $0.0$   \\
\hline
\end{tabular}


In [17]:
print(tabulate.tabulate(exps_cp["Prior"]["fid_values"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))


\begin{tabular}{ll}
\hline
                     & SNGAN   \\
\hline
 GAN                 & $6.5$   \\
 w/o constr.         & $0.0$   \\
 dis                 & $0.0$   \\
 cluster v0          & $0.0$   \\
 cluster (d)         & $0.0$   \\
 cluster v1          & $0.0$   \\
 cluster v1 (d)      & $0.0$   \\
 cluster v2          & $0.0$   \\
 cluster v2 (d)      & $0.0$   \\
 cluster resnet34    & $0.0$   \\
 cluster resnet34 v1 & $0.0$   \\
 cluster resnet34 v2 & $0.0$   \\
 MMD                 & $0.0$   \\
 MMD (d)             & $0.0$   \\
 CMD                 & $0.0$   \\
 CMD (d)             & $0.0$   \\
 MMD resnet34        & $0.0$   \\
 CMD resnet34        & $0.0$   \\
 resnet34            & $6.8$   \\
\hline
\end{tabular}


\begin{tabular}{lllllllll}
\hline
                     & DCGAN   & WGAN-GP   & SNGAN   & SNGAN (R)   & DCGAN (C)   & SNGAN (C)   & WGAN-GP (C)   & SAGAN   \\
\hline
 GAN, 10k            & $35.9$  & $33.1$    & $25.7$  & $18.3$      & $47.6$      & $14.0$      & $24.5$        & $14.8$  \\
 GAN, 1k             & $62.4$  & $59.6$    & $53.0$  & $0.0$       & $47.5$      & $0.0$       & $0.0$         & $0.0$   \\
 w/o constr.         & $85.2$  & $72.2$    & $35.9$  & $25.4$      & $47.5$      & $13.1$      & $42.9$        & $12.8$  \\
 dis                 & $63.7$  & $62.2$    & $55.6$  & $0.0$       & $47.5$      & $0.0$       & $0.0$         & $0.0$   \\
 cluster v0          & $93.4$  & $59.5$    & $53.3$  & $20.3$      & $51.4$      & $0.0$       & $31.1$        & $0.0$   \\
 cluster (d)         & $71.2$  & $97.3$    & $64.0$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster v1          & $95.2$  & $59.6$    & $55.6$  & $31.2$      & $0.0$       & $0.0$       & $59.3$        & $0.0$   \\
 cluster v1 (d)      & $91.3$  & $89.8$    & $66.2$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster v2          & $82.5$  & $62.0$    & $52.1$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster v2 (d)      & $76.3$  & $63.0$    & $51.9$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster resnet34    & $96.4$  & $60.4$    & $50.5$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster resnet34 v1 & $92.4$  & $58.8$    & $50.9$  & $15.1$      & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster resnet34 v2 & $84.3$  & $62.3$    & $51.6$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 MMD                 & $107.8$ & $94.4$    & $64.2$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 MMD (d)             & $110.1$ & $95.5$    & $62.5$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 CMD                 & $66.0$  & $105.0$   & $116.4$ & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 CMD (d)             & $61.5$  & $59.5$    & $50.9$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 MMD resnet34        & $111.9$ & $95.6$    & $64.1$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 CMD resnet34        & $67.2$  & $56.0$    & $51.2$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 resnet34            & $42.6$  & $30.8$    & $22.7$  & $13.8$      & $41.9$      & $12.1$      & $27.4$        & $12.5$  \\
\hline
\end{tabular}
\begin{tabular}{lllllllll}
\hline
                     & DCGAN   & WGAN-GP   & SNGAN   & SNGAN (R)   & DCGAN (C)   & SNGAN (C)   & WGAN-GP (C)   & SAGAN   \\
\hline
 GAN, 10k            & $35.9$  & $33.1$    & $25.7$  & $18.3$      & $47.6$      & $14.0$      & $24.5$        & $14.8$  \\
 GAN, 1k             & $62.4$  & $59.6$    & $53.0$  & $0.0$       & $47.5$      & $0.0$       & $0.0$         & $0.0$   \\
 w/o constr.         & $0.0$   & $0.0$     & $0.0$   & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 dis                 & $62.4$  & $61.1$    & $58.9$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster v0          & $65.5$  & $63.0$    & $56.2$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster (d)         & $59.9$  & $63.5$    & $54.9$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster v1          & $61.5$  & $60.5$    & $54.0$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster v1 (d)      & $92.2$  & $0.0$     & $65.6$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster v2          & $64.5$  & $64.3$    & $54.3$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster v2 (d)      & $59.8$  & $60.7$    & $50.1$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster resnet34    & $65.1$  & $63.2$    & $55.8$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster resnet34 v1 & $61.7$  & $70.3$    & $55.1$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 cluster resnet34 v2 & $65.9$  & $62.9$    & $53.6$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 MMD                 & $62.1$  & $61.0$    & $54.8$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 MMD (d)             & $61.9$  & $62.5$    & $55.9$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 CMD                 & $65.8$  & $91.6$    & $56.8$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 CMD (d)             & $62.6$  & $59.7$    & $51.3$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 MMD resnet34        & $61.2$  & $61.1$    & $55.6$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 CMD resnet34        & $57.4$  & $57.8$    & $50.6$  & $0.0$       & $0.0$       & $0.0$       & $0.0$         & $0.0$   \\
 resnet34            & $32.4$  & $31.5$    & $23.8$  & $16.7$      & $42.0$      & $12.9$      & $23.9$        & $13.6$  \\
\hline
\end{tabular}